In [ ]:
import os
import geopandas as gpd

In [ ]:
os.getcwd()

In [ ]:
lakes_extent_file_path = os.path.join(os.getcwd(), 'amundsen_sub_glacial_lake_mask.gpkg')
lakes_extent_df = gpd.read_file(lakes_extent_file_path)

In [ ]:
lakes_extent_df.head()